In [1]:
import uuid 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BronzePessoas") \
    .config("spark.extraListeners", "io.openlineage.spark.agent.OpenLineageSparkListener") \
    .config("spark.openlineage.columnLineage.datasetLineageEnabled", "true") \
    .config("spark.openlineage.transport.type", "http") \
    .config("spark.openlineage.transport.url", "http://api:5000") \
    .config("spark.openlineage.namespace", "spark_integration") \
    .config("spark.openlineage.parentJobName", "customer-job") \
    .config("spark.openlineage.parentRunId", uuid.uuid4().__str__()) \
    .getOrCreate()

25/06/19 18:11:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

# Inicializar Spark
spark = SparkSession.builder.appName("CriarPessoas").getOrCreate()

# Lista de nomes de clientes (extraída dos seus dados)
nomes = [
    "João",
    "Maria",
    "Lucas",
    "Carlos",
    "Ana"
]

# Criar DataFrame de nomes únicos
df_nomes = spark.createDataFrame([(nome,) for nome in nomes], ["customer_name"])

# Adicionar id_customer usando dense_rank
window_spec = Window.orderBy("customer_name")
df_pessoas = df_nomes.withColumn("id_customer", dense_rank().over(window_spec))

# Reordenar colunas
df_pessoas = df_pessoas.select("id_customer", "customer_name")

25/06/19 18:11:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# Caminho para o bucket MinIO (formato S3)
delta_path = "s3a://bronze/pessoas"

# Escrever como tabela Delta
df_pessoas.write.format("delta").mode("overwrite").save(delta_path)

25/06/19 18:11:41 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/06/19 18:11:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/19 18:11:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/19 18:11:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/19 18:11:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/06/19 18:11:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
                                

In [4]:
spark.stop()